<a href="https://colab.research.google.com/github/jadiribnahasan/Pattern-Lab/blob/main/Model_Evaluation_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/My Drive/Pattern Lab - 203sem

Mounted at /content/gdrive
/content/gdrive/My Drive/Pattern Lab - 203sem


In [2]:
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [3]:
Xtrain = np.load(f'./final datas/Xtrain.npy')
Ytrain = np.load(f'./final datas/Ytrain.npy')
Xtest = np.load(f'./final datas/Xtest.npy')
Ytest = np.load(f'./final datas/Ytest.npy')

with open('index_dict.pickle', 'rb') as handle:
    index_dict = pickle.load(handle)
index_dict

{'CKSNAP1': [164, 195],
 'CKSNAP3': [196, 259],
 'CKSNAP5': [260, 355],
 'CKSNAP7': [356, 483],
 'CKSNAP9': [484, 643],
 'DAC7': [644, 685],
 'EIIP': [686, 726],
 'ENAC10': [875, 1002],
 'ENAC5': [727, 874],
 'Kmer1': [1003, 1006],
 'Kmer2': [1007, 1022],
 'Kmer3': [1023, 1086],
 'Kmer4': [1087, 1342],
 'Kmer5': [1343, 2366],
 'NCP': [2367, 2489],
 'PseEIIP': [2490, 2553],
 'TAC7': [2554, 2567],
 'binary': [0, 163]}

In [4]:
random.seed(13)

features = {1 : 'CKSNAP1', 2 : 'CKSNAP3', 3 : 'CKSNAP5', 4 : 'CKSNAP7', 5 : 'CKSNAP9', 6 : 'DAC7', 7 : 'EIIP', 8 : 'ENAC5', 9 : 'ENAC10', 10 : 'Kmer1', 11 : 'Kmer2', 12 : 'Kmer3', 13 : 'Kmer4', 14 : 'Kmer5', 15 : 'NCP', 16 : 'PseEIIP', 17 : 'TAC7', 18 : 'binary'}

subgrp1 = [features[random.randint(1,18)] for i in range(5)]
subgrp2 = [features[random.randint(1,18)] for i in range(5)]
subgrp3 = [features[random.randint(1,18)] for i in range(5)]
subgrp4 = [features[random.randint(1,18)] for i in range(5)]
subgrp5 = [features[random.randint(1,18)] for i in range(5)]


X1 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp1], axis = 1)
X2 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp2], axis = 1)
X3 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp3], axis = 1)
X4 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp4], axis = 1)
X5 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp5], axis = 1)

Xs = [X1, X2, X3, X4, X5]


In [5]:
subgroups = [subgrp1, subgrp2, subgrp3, subgrp4, subgrp5]
for group in subgroups:
  print(group)

['ENAC10', 'Kmer1', 'DAC7', 'ENAC5', 'CKSNAP9']
['ENAC5', 'DAC7', 'CKSNAP9', 'CKSNAP5', 'binary']
['EIIP', 'Kmer1', 'CKSNAP1', 'Kmer5', 'CKSNAP9']
['CKSNAP1', 'ENAC10', 'CKSNAP9', 'CKSNAP5', 'ENAC10']
['NCP', 'Kmer5', 'CKSNAP9', 'ENAC10', 'Kmer3']


In [6]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from numpy import mean

kf = KFold(n_splits=5, shuffle = True)

***Logistic Regression***

In [ ]:
lg = LogisticRegression(max_iter=500)

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in logistic regression \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in logistic regression \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in logistic regression \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in logistic regression \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in logistic regression 
 train -> 0.7980092081288873  test -> 0.7621876042847031
subgroup1 sensitivity in logistic regression 
 train -> 0.7992702334345018  test -> 0.7597934723309336
subgroup1 specificity in logistic regression 
 train -> 0.7967227568189822  test -> 0.7643131323899665
subgroup1 mcc in logistic regression 
 train -> 0.5960183875476167  test -> 0.5243348925904534
subgroup2 accuracy in logistic regression 
 train -> 0.8099947473175437  test -> 0.7722143378402662
subgroup2 sensitivity in logistic regression 
 train -> 0.8075766382632619  test -> 0.7695537218256228
subgroup2 specificity in logistic regression 
 train -> 0.8124346857448191  test -> 0.7753718773904958
subgroup2 mcc in logistic regression 
 train -> 0.6200189680009975  test -> 0.5448444591143934
subgroup3 accuracy in logistic regression 
 train -> 0.7344932949772112  test -> 0.7090989434928179
subgroup3 sensitivity in logistic regression 
 train -> 0.7527609772715235  test -> 0.728694706736

***XG Boost***

In [ ]:
xgbst = XGBClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    xgbst.fit(X_train, y_train)
    y_pred = xgbst.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    xgbst.fit(X_train, y_train)
    y_pred = xgbst.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in XG boost \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in XG boost \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in XG boost \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in XG boost \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in XG boost 
 train -> 0.8450698955264159  test -> 0.7283302345878668
subgroup1 sensitivity in XG boost 
 train -> 0.8535979293508872  test -> 0.7330034835845263
subgroup1 specificity in XG boost 
 train -> 0.8365476458290532  test -> 0.7238065627711827
subgroup1 mcc in XG boost 
 train -> 0.6902493728721829  test -> 0.45709589442362725
subgroup2 accuracy in XG boost 
 train -> 0.8669412077868094  test -> 0.7711226259759215
subgroup2 sensitivity in XG boost 
 train -> 0.873760420053473  test -> 0.7712452228291922
subgroup2 specificity in XG boost 
 train -> 0.8600928227065093  test -> 0.771215200644976
subgroup2 mcc in XG boost 
 train -> 0.7339209084590337  test -> 0.5423043098574687
subgroup3 accuracy in XG boost 
 train -> 0.854685789396093  test -> 0.7667873160463674
subgroup3 sensitivity in XG boost 
 train -> 0.8629558740602146  test -> 0.7623726192045033
subgroup3 specificity in XG boost 
 train -> 0.8464279969428464  test -> 0.7713196715551565
subgroup3 mcc i

***Ada Boost***

In [ ]:
ab = AdaBoostClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in Ada boost \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in Ada boost \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in Ada boost \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in Ada boost \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in Ada boost 
 train -> 0.7560944299784872  test -> 0.7036829231621138
subgroup1 sensitivity in Ada boost 
 train -> 0.757843706845335  test -> 0.7079571819698474
subgroup1 specificity in Ada boost 
 train -> 0.7542984541052062  test -> 0.6991608409694111
subgroup1 mcc in Ada boost 
 train -> 0.5122387038428017  test -> 0.40752185385847006
subgroup2 accuracy in Ada boost 
 train -> 0.7871757053233679  test -> 0.7445870967505346
subgroup2 sensitivity in Ada boost 
 train -> 0.7810077146017175  test -> 0.7311334837486235
subgroup2 specificity in Ada boost 
 train -> 0.7933369521220605  test -> 0.7577631601935177
subgroup2 mcc in Ada boost 
 train -> 0.5744027896193985  test -> 0.4891008186819474
subgroup3 accuracy in Ada boost 
 train -> 0.7862948779353175  test -> 0.7421418381978137
subgroup3 sensitivity in Ada boost 
 train -> 0.7865480723067779  test -> 0.7441977047418804
subgroup3 specificity in Ada boost 
 train -> 0.7860130477400927  test -> 0.7405849192341247
su

***Random Forest***

In [ ]:
rf = RandomForestClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in Random forest \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in Random forest \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in Random forest \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in Random forest \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in Random forest 
 train -> 1.0  test -> 0.6923132776659295
subgroup1 sensitivity in Random forest 
 train -> 1.0  test -> 0.6974055415606969
subgroup1 specificity in Random forest 
 train -> 1.0  test -> 0.687639367223958
subgroup1 mcc in Random forest 
 train -> 1.0  test -> 0.38532021540917116
subgroup2 accuracy in Random forest 
 train -> 1.0  test -> 0.7185814713356876
subgroup2 sensitivity in Random forest 
 train -> 1.0  test -> 0.7269207505472675
subgroup2 specificity in Random forest 
 train -> 1.0  test -> 0.7099865701281234
subgroup2 mcc in Random forest 
 train -> 1.0  test -> 0.4369954490443602
subgroup3 accuracy in Random forest 
 train -> 1.0  test -> 0.6974582952866065
subgroup3 sensitivity in Random forest 
 train -> 1.0  test -> 0.6998728680806237
subgroup3 specificity in Random forest 
 train -> 1.0  test -> 0.6954831665062841
subgroup3 mcc in Random forest 
 train -> 1.0  test -> 0.3951562203428419
subgroup4 accuracy in Random forest 
 train -> 1.

***SVC***

In [ ]:
svc = SVC()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in svc \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in svc \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in svc \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in svc \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in svc 
 train -> 0.8441900081644708  test -> 0.7502726529295064
subgroup1 sensitivity in svc 
 train -> 0.8572267581830767  test -> 0.7585940289872679
subgroup1 specificity in svc 
 train -> 0.8310956277315483  test -> 0.7423943195644365
subgroup1 mcc in svc 
 train -> 0.6886038150144722  test -> 0.5011262074020597
subgroup2 accuracy in svc 
 train -> 0.9117689892724681  test -> 0.7662555786586283
subgroup2 sensitivity in svc 
 train -> 0.9206440464144471  test -> 0.7677682510447441
subgroup2 specificity in svc 
 train -> 0.9028804681397002  test -> 0.7647904746199963
subgroup2 mcc in svc 
 train -> 0.8236952537401836  test -> 0.5326389789933934
subgroup3 accuracy in svc 
 train -> 0.7696367670347061  test -> 0.7218294699861748
subgroup3 sensitivity in svc 
 train -> 0.7884544787875216  test -> 0.7464192648348646
subgroup3 specificity in svc 
 train -> 0.7507748094900097  test -> 0.697456198545549
subgroup3 mcc in svc 
 train -> 0.5396399936523814  test -> 0.4442716

***Independent Test(Logistic Regression)***

In [8]:
lg = LogisticRegression(max_iter=500)

acc = []
sensitivity = []
specificity = []
mcc = []
lg.fit(Xtrain, Ytrain)
y_pred = lg.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7992424242424242 
 sensitivity -> 0.7727272727272727 
 specificity -> 0.8257575757575758 
 mcc -> 0.5993281610305964


***Independent Test(XG Boost)***

In [9]:
xgbst = XGBClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
xgbst.fit(Xtrain, Ytrain)
y_pred = xgbst.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7916666666666666 
 sensitivity -> 0.803030303030303 
 specificity -> 0.7803030303030303 
 mcc -> 0.5834840459914051


***Independent Test(Ada Boost)***

In [10]:
ab = AdaBoostClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
ab.fit(Xtrain, Ytrain)
y_pred = ab.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7537878787878788 
 sensitivity -> 0.7348484848484849 
 specificity -> 0.7727272727272727 
 mcc -> 0.5079402853927962


***Independent Test(Rand. forest)***

In [11]:
rf = RandomForestClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
rf.fit(Xtrain, Ytrain)
y_pred = rf.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7007575757575758 
 sensitivity -> 0.7272727272727273 
 specificity -> 0.6742424242424242 
 mcc -> 0.4020809181597672


***Independent Test(SVC)***

In [12]:
svc = SVC()

acc = []
sensitivity = []
specificity = []
mcc = []
svc.fit(Xtrain, Ytrain)
y_pred = svc.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.803030303030303 
 sensitivity -> 0.803030303030303 
 specificity -> 0.803030303030303 
 mcc -> 0.6060606060606061
